# Read Geopackages
https://pcjericks.github.io/py-gdalogr-cookbook/

https://gdal.org/python/

In [2]:
from osgeo import ogr

In [26]:
def printDriverNames():
    cnt = ogr.GetDriverCount()
    drivers = []  # Empty List

    for i in range(cnt):
        driver = ogr.GetDriver(i)
        driverName = driver.GetName()
        if not driverName in drivers:
            drivers.append(driverName)

    drivers.sort() # Sorting the messy list of ogr drivers

    for driver in drivers:
        print(driver)

def getGeomTypeName(geometryType):
    if 0 == geometryType:
        return "Unknown"
    if 1 == geometryType:
        return "Point"
    if 2 == geometryType:
        return "Linestring"
    if 3 == geometryType:
        return "Polygon"
    if 4 == geometryType:
        return "Multipoint"
    if 5 == geometryType:
        return "Multilinestring"
    if 6 == geometryType:
        return "Multipolygon"
    if 7 == geometryType:
        return "Geometrycollection"
    return "<Not specified>"
    
def printLayerMetadata(layer):
    print(layer.GetName())
    print(getGeomTypeName(layer.GetGeomType()))
    print(layer.GetFIDColumn())
    print(layer.GetGeometryColumn())
        
def printLayerDefinition(layer):
    layerDefinition = layer.GetLayerDefn()
    for fieldIndex in range(layerDefinition.GetFieldCount()):
        print(layerDefinition.GetFieldDefn(fieldIndex).GetName())

In [54]:
#printDriverNames()

driverName = "GPKG"
driver = ogr.GetDriverByName(driverName)
if (driver is None):
    print("Geopackage driver is not available!")
    
filePath = "D:/OSM/Deutschland/Dessau/OSM_Dessau_multipolygons.gpkg"
dataSource = ogr.Open(filePath, 0)
layer = dataSource.GetLayer()

#printLayerMetadata(layer)
#printLayerDefinition(layer)

# Collect all geometries
osmIds = []
geometries = ogr.Geometry(ogr.wkbGeometryCollection)
lastGeoJson = ""
for feature in layer:
    osmIds.append(feature.GetField("osm_id"))
    geometry = feature.GetGeometryRef()
    geometries.AddGeometry(geometry)
    lastGeoJson = geometry.ExportToJson()
layer.ResetReading()

print("Found {0} ids in layer {1}.".format(len(osmIds), layer.GetName()))

# Calculate convex hull
convexHull = geometries.ConvexHull()
print(convexHull)

print("Last geometry is {}".format(lastGeoJson))

layer = None
dataSource = None

Found 666 ids in layer OSM_Dessau_multipolygons.
POLYGON ((12.3225967 51.7331378,12.181572 51.8164557,12.1812612 51.8167569,12.228545 51.8781688,12.2288099 51.8784858,12.2291469 51.8787907,12.2309409 51.8801158,12.2321074 51.8805456,12.2325593 51.8807055,12.2330843 51.8808465,12.2336462 51.8809195,12.2354079 51.8810225,12.2369793 51.881105,12.2955962 51.8804076,12.2961727 51.8797646,12.3185765 51.8389851,12.3194066 51.8372496,12.3428475 51.7863173,12.3473881 51.7718427,12.3475688 51.7708347,12.3518038 51.7368747,12.3517513 51.7366166,12.3514688 51.7357955,12.3509306 51.7353011,12.3484465 51.7338598,12.3225967 51.7331378))
Last geometry is { "type": "MultiPolygon", "coordinates": [ [ [ [ 12.2434354, 51.8263771 ], [ 12.2434357, 51.826455 ], [ 12.2433907, 51.8264551 ], [ 12.2433903, 51.8263771 ], [ 12.2434354, 51.8263771 ] ] ] ] }
